# Import Modules and Data

In [1]:
# Load modules
import sys
import os
os.chdir('..')

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

from functions import cvmodeleval,samplecv, trainsampling

In [2]:
# Set memory limit on GPU to keep it from freezing up when fitting TensorFlow models later
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 3 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], \
                                                                [tf.config.experimental.\
                                                                 VirtualDeviceConfiguration\
                                                                 (memory_limit=1024 * 3.5)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# Load training filename dataframe
df = pd.read_csv('data/processed/driver_image_list_processed.csv')

In [4]:
# Labeled and unlabeled test filename dataframes
df_test_labeled = pd.read_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/processed/labeled_test_df.csv')
df_test = pd.read_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/raw/test_filenames.csv')

In [5]:
# Call custom function to over/undersample classes occurance by subject so dataset is completely balanced.
df = trainsampling(df, samples=80, random_state=42)

In [6]:
# Define path variables for data
train_path = 'C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/raw/imgs/train'
labeled_test_path = 'C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/raw/imgs/testlabeled'
unlabeled_test_path = 'C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/raw/imgs/test'

4 subjects were chosen from the training data to be used for validation during model training. These subjects represent one woman and man with dark skin and one woman and man with light skin. This is to help balance any potential racial bias in the model.

In [7]:
# Define list of validation subjects
val_subjects = ['p056', 'p050', 'p041', 'p016']

In [8]:
# Define training and validation datasets
df_train = df[~df['subject'].isin(val_subjects)]
df_val = df[df['subject'].isin(val_subjects)]

In [9]:
# Shuffle the dataframes
df_train = df_train.sample(frac=1, random_state=42)
df_val = df_val.sample(frac=1, random_state=42)

## Pre-Processing and Data-Loading

In [10]:
# Define ImageDataGenerators for training and test/validation data. Generators include randomized preprocessing for 
# called out parameters. Test_dgen will be used for both validation data and test data.

train_dgen = ImageDataGenerator(samplewise_center=True,
                                rescale=1./255,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                channel_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                brightness_range=[0.5, 1.5])

test_dgen = ImageDataGenerator(samplewise_center=True,
                               rescale=1./255)

In [11]:
# Load training, validation, and test data
train = train_dgen.flow_from_dataframe(df_train,
                                       x_col='imgpath',
                                       y_col='classname',
                                       batch_size=16, 
                                       target_size=(227,227),
                                       shuffle=True)

val = test_dgen.flow_from_dataframe(df_val,
                                    x_col='imgpath',
                                    y_col='classname',
                                    target_size=(227,227),
                                    shuffle=False)

test_labeled = test_dgen.flow_from_dataframe(df_test_labeled,
                                             x_col='filename',
                                             y_col='classname',
                                             target_size=(227,227),
                                             shuffle=False)

test = test_dgen.flow_from_dataframe(df_test,
                                     x_col='filename',
                                     y_col='class',
                                     target_size=(227,227),
                                     shuffle=False)


Found 17600 validated image filenames belonging to 10 classes.
Found 3200 validated image filenames belonging to 10 classes.
Found 200 validated image filenames belonging to 10 classes.
Found 79726 validated image filenames belonging to 1 classes.


# Model1 - AlexNet Architecture

Architecture below is based off of AlexNet

In [10]:
# Define function for initializing model
def alexNet_arch(opt):
    model = Sequential()
    model.add(Conv2D(99,
                  kernel_size=11,
                  strides=4,
                  padding='valid',
                  input_shape=(227, 227, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(3,
                         strides=2,
                         padding='valid'))
    model.add(Conv2D(256,
                      kernel_size=5,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(3,
                        strides=2,
                        padding='valid'))
    model.add(Conv2D(384,
                      kernel_size=3,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(384,
                      kernel_size=3,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256,
                      kernel_size=3,
                      strides=1,
                      padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(3,
                         strides=2,
                         padding='valid'))
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
# Define optimizer variable
opt = RMSprop(learning_rate=0.0001)

In [12]:
# Compile model_1
model_1 = alexNet_arch(opt)

In [15]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/weights/model1_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=20, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [16]:
%%time
# Train model_1
model_1.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/100
110/110 [==============================] - 71s 643ms/step - loss: 4.4653 - accuracy: 0.1148 - val_loss: 2.3744 - val_accuracy: 0.1000
Epoch 2/100
110/110 [==============================] - 64s 584ms/step - loss: 2.6455 - accuracy: 0.0949 - val_loss: 2.3084 - val_accuracy: 0.1000
Epoch 3/100
110/110 [==============================] - 56s 512ms/step - loss: 2.4212 - accuracy: 0.1159 - val_loss: 2.3324 - val_accuracy: 0.1000
Epoch 4/100
110/110 [==============================] - 62s 560ms/step - loss: 2.3981 - accuracy: 0.1267 - val_loss: 2.2477 - val_accuracy: 0.1603
Epoch 5/100
110/110 [==============================] - 56s 511ms/step - loss: 2.2667 - accuracy: 0.1739 - val_loss: 2.3617 - val_accuracy: 0.1003
Epoch 6/100
110/110 [==============================] - 60s 546ms/step - loss: 2.1387 - accuracy: 0.1972 - val_loss: 2.0907 - val_accuracy: 0.2722
Epoch 7/100
110/110 [=========

110/110 [==============================] - 57s 516ms/step - loss: 0.6652 - accuracy: 0.7898 - val_loss: 2.2284 - val_accuracy: 0.6678
Epoch 56/100
110/110 [==============================] - 60s 548ms/step - loss: 0.5888 - accuracy: 0.8040 - val_loss: 1.3704 - val_accuracy: 0.6128
Epoch 57/100
110/110 [==============================] - 59s 535ms/step - loss: 0.5953 - accuracy: 0.8057 - val_loss: 1.2898 - val_accuracy: 0.5972
Epoch 58/100
110/110 [==============================] - 60s 547ms/step - loss: 0.5930 - accuracy: 0.8000 - val_loss: 1.8623 - val_accuracy: 0.5825
Epoch 59/100
110/110 [==============================] - 57s 521ms/step - loss: 0.5794 - accuracy: 0.8074 - val_loss: 1.1326 - val_accuracy: 0.6966
Epoch 60/100
110/110 [==============================] - 57s 516ms/step - loss: 0.5962 - accuracy: 0.8062 - val_loss: 3.8987 - val_accuracy: 0.4913
Epoch 61/100
110/110 [==============================] - 57s 514ms/step - loss: 0.6026 - accuracy: 0.8045 - val_loss: 1.1211 - val_a

In [17]:
# Save model predictions of validation data
validation_predictions = model_1.predict(val)

In [18]:
# Convert to a dataframe with original labeles
validation_predictions = pd.DataFrame(validation_predictions, 
                                         columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
validation_predictions = df_val.reset_index(drop=True).join(validation_predictions)

,subject,classname,img,imgpath,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,p050,c9,img_37057.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,6.714016e-02,9.080641e-03,3.276221e-02,9.708376e-04,3.149434e-03,8.852060e-02,1.004019e-02,4.595437e-01,2.119597e-01,1.168326e-01
1,p056,c1,img_70214.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,9.916137e-06,9.999266e-01,2.778533e-09,2.121129e-05,9.715735e-09,8.799989e-11,3.782401e-05,7.553241e-09,2.146324e-06,2.380554e-06
2,p050,c7,img_89317.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,4.676285e-03,6.653389e-04,6.193582e-03,3.859256e-05,5.127674e-04,9.498447e-03,2.100115e-03,8.335724e-01,1.233497e-01,1.939294e-02
3,p041,c1,img_9714.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,2.411511e-07,9.999948e-01,2.486746e-11,2.996047e-07,3.825119e-11,7.392742e-14,4.569526e-06,2.724793e-11,6.434313e-08,3.570090e-08
4,p016,c2,img_46158.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,8.731217e-07,3.443508e-07,9.989744e-01,6.951326e-11,8.678994e-08,5.064950e-07,7.010580e-05,1.246347e-04,8.061368e-04,2.294997e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,p041,c3,img_78650.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,3.715247e-10,5.769833e-12,6.908332e-29,1.000000e+00,5.136259e-13,3.894515e-23,6.313904e-23,6.123699e-26,1.471512e-17,3.946067e-14
3196,p041,c4,img_83455.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,1.574739e-06,4.375948e-10,6.848797e-11,7.733332e-07,9.999092e-01,8.031368e-09,9.146721e-10,4.092025e-10,3.493802e-06,8.497512e-05
3197,p041,c6,img_58453.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,1.789286e-10,3.245966e-06,9.168875e-06,2.270906e-12,9.345583e-07,1.655015e-10,9.876809e-01,3.072587e-09,1.230577e-02,3.856044e-08
3198,p041,c0,img_51874.jpg,C:/Users/Dylan/Desktop/Data Science/Projects/D...,6.973295e-01,2.495644e-03,8.154131e-04,3.410958e-03,6.379608e-03,1.482557e-02,4.676780e-04,1.062956e-02,5.146125e-02,2.121847e-01


In [19]:
# Save validation predictions
validation_predictions.to_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/validation_predictions/model_1_validation_predictions.csv')

In [71]:
# Evaluate validation data with best coefficients
model_1_val_metrics = model_1.evaluate(val)

  ...
    to  
  ['...']
100/100 [==============================] - 20s 201ms/step - loss: 0.9765 - accuracy: 0.6913


In [73]:
model_1_val_metrics = pd.DataFrame({'Cross Entropy Loss':[model_1_val_metrics[0]], 
                                    'Accuracy':[model_1_val_metrics[1]]})
model_1_val_metrics.to_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/metrics/model_1_val_metrics.csv', index=False)

[0.9764609852433205, 0.69125]

In [25]:
model_1_val_metrics

,Cross Entropy Loss,Accuracy
0,0.976461,0.69125


0.9765 cross-entropy loss and 0.6913 accuracy are not a bad start

### Model 2 - Xception Transfer Learning

Using a method desribed at this url (https://www.analyticsvidhya.com/blog/2020/08/top-4-pre-trained-models-for-image-classification-with-python-code/), I loaded a pre-trained Xception model through keras with weights optimized for imagenet. I made all of the existing layers non-trainable and then added a few trainable layers which will be fit to the data.

In [12]:
base_model_2 = tf.keras.applications.Xception(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_2.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_2.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_2 = tf.keras.models.Model(base_model_2.input, x)
model_2.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_2.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 227, 227, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 113, 113, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 113, 113, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 113, 113, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [13]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/weights/model2_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=20, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [14]:
model_2.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/100
110/110 [==============================] - 71s 647ms/step - loss: 2.8249 - accuracy: 0.1415 - val_loss: 2.1583 - val_accuracy: 0.2169
Epoch 2/100
110/110 [==============================] - 60s 549ms/step - loss: 2.3187 - accuracy: 0.1727 - val_loss: 2.2208 - val_accuracy: 0.2288
Epoch 3/100
110/110 [==============================] - 62s 560ms/step - loss: 2.1792 - accuracy: 0.2364 - val_loss: 2.4904 - val_accuracy: 0.1937
Epoch 4/100
110/110 [==============================] - 60s 548ms/step - loss: 2.1252 - accuracy: 0.2557 - val_loss: 2.3664 - val_accuracy: 0.2147
Epoch 5/100
110/110 [==============================] - 59s 535ms/step - loss: 2.0460 - accuracy: 0.3051 - val_loss: 2.4786 - val_accuracy: 0.2256
Epoch 6/100
110/110 [==============================] - 58s 526ms/step - loss: 1.9675 - accuracy: 0.3210 - val_loss: 2.5725 - val_accuracy: 0.2037
Epoch 7/100
110/110 [=========

In [15]:
# Save model predictions of validation data
validation_predictions = model_2.predict(val)

# Convert to a dataframe with original labels
validation_predictions = pd.DataFrame(validation_predictions, 
                                         columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
validation_predictions = df_val.reset_index(drop=True).join(validation_predictions)

In [16]:
# Save validation predictions
validation_predictions.to_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/validation_predictions/model_2_validation_predictions.csv')

In [17]:
# Evaluate validation data with best coefficients
model_2_val_metrics = model_2.evaluate(val)

  ...
    to  
  ['...']
100/100 [==============================] - 19s 192ms/step - loss: 2.1583 - accuracy: 0.2169


In [21]:
model_2_val_metrics = pd.DataFrame({'Cross Entropy Loss':[model_2_val_metrics[0]], 
                                    'Accuracy':[model_2_val_metrics[1]]})
model_2_val_metrics.to_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/metrics/model_2_val_metrics.csv', index=False)

In [22]:
model_2_val_metrics

,Cross Entropy Loss,Accuracy
0,2.15828,0.216875


### Model 3 - VGG16 Transfer-Learning

In [12]:
base_model_3 = tf.keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))
for layer in base_model_3.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model_3.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model_3 = tf.keras.models.Model(base_model_3.input, x)
model_3.compile(optimizer = RMSprop(0.0001), loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_3.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [21]:
# Designate ModelCheckpoint and EarlyStopping callbacks_list
checkpoint = ModelCheckpoint('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/weights/model3_weights.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

earlystop = EarlyStopping(monitor='val_loss', min_delta=0.025, patience=50, restore_best_weights=True)

callbacks_list = [checkpoint, earlystop]

In [14]:
model_3.fit(train,
            epochs=100,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/100
110/110 [==============================] - 63s 569ms/step - loss: 2.5059 - accuracy: 0.1324 - val_loss: 2.1846 - val_accuracy: 0.1606
Epoch 2/100
110/110 [==============================] - 59s 541ms/step - loss: 2.2511 - accuracy: 0.1653 - val_loss: 2.0830 - val_accuracy: 0.2734
Epoch 3/100
110/110 [==============================] - 59s 538ms/step - loss: 2.1532 - accuracy: 0.2284 - val_loss: 1.9477 - val_accuracy: 0.2266
Epoch 4/100
110/110 [==============================] - 59s 535ms/step - loss: 2.1002 - accuracy: 0.2295 - val_loss: 1.8449 - val_accuracy: 0.3663
Epoch 5/100
110/110 [==============================] - 59s 537ms/step - loss: 1.9959 - accuracy: 0.2886 - val_loss: 1.6320 - val_accuracy: 0.5341
Epoch 6/100
110/110 [==============================] - 60s 543ms/step - loss: 1.9342 - accuracy: 0.3034 - val_loss: 1.5335 - val_accuracy: 0.6256
Epoch 7/100
110/110 [=========

110/110 [==============================] - 58s 527ms/step - loss: 0.8851 - accuracy: 0.7000 - val_loss: 0.8380 - val_accuracy: 0.7188
Epoch 56/100
110/110 [==============================] - 58s 528ms/step - loss: 0.8359 - accuracy: 0.7199 - val_loss: 0.9066 - val_accuracy: 0.6913
Epoch 57/100
110/110 [==============================] - 60s 544ms/step - loss: 0.8708 - accuracy: 0.7142 - val_loss: 0.8984 - val_accuracy: 0.6897
Epoch 58/100
110/110 [==============================] - 59s 539ms/step - loss: 0.8583 - accuracy: 0.7125 - val_loss: 0.8167 - val_accuracy: 0.7334
Epoch 59/100
110/110 [==============================] - 59s 534ms/step - loss: 0.8788 - accuracy: 0.7142 - val_loss: 0.8421 - val_accuracy: 0.7034
Epoch 60/100
110/110 [==============================] - 60s 547ms/step - loss: 0.8753 - accuracy: 0.7028 - val_loss: 0.8929 - val_accuracy: 0.6784
Epoch 61/100
110/110 [==============================] - 63s 571ms/step - loss: 0.8081 - accuracy: 0.7312 - val_loss: 0.9423 - val_a

I renamed the weight file for this first run to model3_weights_first_run.hdf5

In [22]:
# Fit the model further and see if it improves
model_3.fit(train,
            epochs=200,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/200
110/110 [==============================] - 59s 537ms/step - loss: 0.9516 - accuracy: 0.6835 - val_loss: 0.9357 - val_accuracy: 0.6862
Epoch 2/200
110/110 [==============================] - 59s 538ms/step - loss: 0.9273 - accuracy: 0.6972 - val_loss: 0.8685 - val_accuracy: 0.7056
Epoch 3/200
110/110 [==============================] - 57s 519ms/step - loss: 0.9531 - accuracy: 0.7017 - val_loss: 0.9524 - val_accuracy: 0.6603
Epoch 4/200
110/110 [==============================] - 58s 528ms/step - loss: 0.8974 - accuracy: 0.6977 - val_loss: 0.8452 - val_accuracy: 0.7459
Epoch 5/200
110/110 [==============================] - 56s 512ms/step - loss: 0.9220 - accuracy: 0.6932 - val_loss: 0.8573 - val_accuracy: 0.7159
Epoch 6/200
110/110 [==============================] - 57s 518ms/step - loss: 0.9031 - accuracy: 0.6949 - val_loss: 0.8959 - val_accuracy: 0.6862
Epoch 7/200
110/110 [=========

110/110 [==============================] - 57s 516ms/step - loss: 0.7343 - accuracy: 0.7585 - val_loss: 0.8584 - val_accuracy: 0.7191
Epoch 56/200
110/110 [==============================] - 56s 512ms/step - loss: 0.7310 - accuracy: 0.7688 - val_loss: 0.8474 - val_accuracy: 0.7191
Epoch 57/200
110/110 [==============================] - 57s 519ms/step - loss: 0.7133 - accuracy: 0.7727 - val_loss: 0.9263 - val_accuracy: 0.7231
Epoch 58/200
110/110 [==============================] - 56s 513ms/step - loss: 0.7189 - accuracy: 0.7597 - val_loss: 0.9608 - val_accuracy: 0.6919
Epoch 59/200
110/110 [==============================] - 57s 520ms/step - loss: 0.7463 - accuracy: 0.7716 - val_loss: 0.9073 - val_accuracy: 0.7328
Epoch 60/200
110/110 [==============================] - 56s 512ms/step - loss: 0.7278 - accuracy: 0.7631 - val_loss: 0.9813 - val_accuracy: 0.7300
Epoch 61/200
110/110 [==============================] - 57s 517ms/step - loss: 0.6926 - accuracy: 0.7744 - val_loss: 0.9074 - val_a

I renamed the weight file for this second run to model3_weights_second_run.hdf5

Change the learning rate from 0.0001 to 0.00001, reload the weights from the first run, and then see if it improves.

In [39]:
# Change learning rate
K.set_value(model_3.optimizer.learning_rate, 0.00001)

In [40]:
# Load weights from first run
model_3.load_weights('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/weights/model3_weights_first_run.hdf5')

In [41]:
# Update callbacks_list. Remove earlystop. Have run for 600 epochs overnight.
checkpoint = ModelCheckpoint('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/weights/model3_weights_third_run.hdf5',
                              mode='min',
                              monitor='val_loss',
                              save_best_only=True)

callbacks_list = [checkpoint]

In [ ]:
# Try improving with learning rate=0.00001
model_3.fit(train,
            epochs=600,
            steps_per_epoch=110,
            validation_data=val,
            callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/600
 51/110 [============>.................] - ETA: 20s - loss: 0.8827 - accuracy: 0.7243

In [15]:
# Save model predictions of validation data
validation_predictions = model_3.predict(val)

# Convert to a dataframe with original labels
validation_predictions = pd.DataFrame(validation_predictions, 
                                         columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
validation_predictions = df_val.reset_index(drop=True).join(validation_predictions)

In [16]:
# Save validation predictions
validation_predictions.to_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/validation_predictions/model_3_validation_predictions.csv')

In [17]:
# Evaluate validation data with best coefficients
model_3_val_metrics = model_3.evaluate(val)

  ...
    to  
  ['...']
100/100 [==============================] - 20s 197ms/step - loss: 0.7318 - accuracy: 0.7569


In [18]:
model_3_val_metrics = pd.DataFrame({'Cross Entropy Loss':[model_3_val_metrics[0]], 
                                    'Accuracy':[model_3_val_metrics[1]]})
model_3_val_metrics.to_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/metrics/model_3_val_metrics.csv', index=False)

In [19]:
model_3_val_metrics

,Cross Entropy Loss,Accuracy
0,0.731836,0.756875


Getting better!

In [62]:
base_model3 = tf.keras.applications.VGG16(include_top=False,
                                        weights='imagenet',
                                        input_shape=(227,227,3))

In [63]:
for layer in model3.layers:
    layer.trainable = False

In [64]:
from tensorflow.keras import layers

In [65]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(model3.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(10, activation='softmax')(x)

model3 = tf.keras.models.Model(model3.input, x)

model3.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

In [66]:
model3.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 227, 227, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 227, 227, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 113, 113, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 113, 113, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 113, 113, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [67]:
os.getcwd()

'C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\raw\\imgs\\testlabeled\\unsorted'

In [70]:
model3.load_weights('C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\raw\\weights.hdf5')

In [33]:
model3.fit(train,
                epochs=50,
                steps_per_epoch=110,
                validation_data=test,
                callbacks=callbacks_list)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 110 steps, validate for 100 steps
Epoch 1/50
110/110 [==============================] - 73s 667ms/step - loss: 2.5233 - acc: 0.1318 - val_loss: 2.2212 - val_acc: 0.1472
Epoch 2/50
110/110 [==============================] - 73s 666ms/step - loss: 2.2779 - acc: 0.1562 - val_loss: 2.1070 - val_acc: 0.3166
Epoch 3/50
110/110 [==============================] - 73s 663ms/step - loss: 2.1991 - acc: 0.2000 - val_loss: 1.9553 - val_acc: 0.4006
Epoch 4/50
110/110 [==============================] - 68s 621ms/step - loss: 2.0999 - acc: 0.2307 - val_loss: 1.8996 - val_acc: 0.4181
Epoch 5/50
110/110 [==============================] - 71s 644ms/step - loss: 2.0020 - acc: 0.2710 - val_loss: 1.6876 - val_acc: 0.4712
Epoch 6/50
110/110 [==============================] - 65s 587ms/step - loss: 1.9430 - acc: 0.3136 - val_loss: 1.6530 - val_acc: 0.5459
Epoch 7/50
110/110 [==============================] - 65s 590ms/step - loss: 1.8607 - acc: 0.343

In [34]:
validation_predictions = model3.predict(test)

In [35]:
validation_predictions = pd.DataFrame(validation_predictions, 
                                         columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [36]:
classifier_train_data = df_test.reset_index(drop=True).join(validation_predictions)

In [37]:
classifier_train_data

,subject,classname,img,imgpath,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,p050,c9,img_37057.jpg,imgs/train/c9/img_37057.jpg,0.038092,0.047378,0.055674,0.003527,0.011132,0.032342,0.036965,0.002194,0.729606,0.043091
1,p056,c1,img_70214.jpg,imgs/train/c1/img_70214.jpg,0.001814,0.665885,0.046201,0.011587,0.001652,0.002267,0.147198,0.049998,0.066739,0.006660
2,p050,c7,img_89317.jpg,imgs/train/c7/img_89317.jpg,0.004418,0.058346,0.321970,0.004580,0.001134,0.001784,0.017168,0.522277,0.043075,0.025248
3,p041,c1,img_9714.jpg,imgs/train/c1/img_9714.jpg,0.021931,0.579883,0.051502,0.050977,0.004998,0.064224,0.037028,0.003008,0.048320,0.138128
4,p016,c2,img_46158.jpg,imgs/train/c2/img_46158.jpg,0.003152,0.329363,0.136421,0.014852,0.012734,0.014436,0.121248,0.182954,0.082555,0.102285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,p041,c3,img_78650.jpg,imgs/train/c3/img_78650.jpg,0.009461,0.001977,0.004556,0.822273,0.130280,0.014533,0.001842,0.000328,0.008226,0.006525
3196,p041,c4,img_83455.jpg,imgs/train/c4/img_83455.jpg,0.007229,0.000311,0.011610,0.164005,0.793564,0.010024,0.003079,0.000376,0.006493,0.003310
3197,p041,c6,img_58453.jpg,imgs/train/c6/img_58453.jpg,0.005851,0.024534,0.390130,0.028171,0.084743,0.035849,0.285895,0.004593,0.124124,0.016109
3198,p041,c0,img_51874.jpg,imgs/train/c0/img_51874.jpg,0.409633,0.021191,0.047486,0.014772,0.004590,0.097627,0.006294,0.039794,0.050137,0.308477


In [38]:
classifier_train_data.to_csv('../processed/VGG16_classifier_training_data.csv')

In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X = classifier_train_data.drop(columns=['subject', 'classname', 'img', 'imgpath'])
y= classifier_train_data[['classname']]

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [42]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [43]:
params = {'criterion':['gini', 'entropy'],
          'min_samples_split':np.arange(2, 20, 1),
          'min_samples_leaf':np.arange(1, 20, 1)}

In [71]:
clf = RandomForestClassifier(n_jobs=-2)

In [74]:
RandForest = RandomizedSearchCV(clf, params, scoring='accuracy', 
                           n_iter=30)

In [75]:
RandForest.fit(X_train, np.ravel(y_train))

RandomizedSearchCV(estimator=RandomForestClassifier(n_jobs=-2), n_iter=30,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                                        'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19])},
                   scoring='accuracy')

In [76]:
print(RandForest.best_params_)

{'min_samples_split': 5, 'min_samples_leaf': 2, 'criterion': 'entropy'}


In [77]:
from sklearn.metrics import classification_report

In [78]:
y_pred = RandForest.predict(X_test)

In [79]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          c0       0.88      0.97      0.93        77
          c1       0.96      0.96      0.96        82
          c2       0.92      0.92      0.92        78
          c3       0.99      0.95      0.97        87
          c4       0.98      0.99      0.98        89
          c5       1.00      1.00      1.00        76
          c6       0.89      0.95      0.92        78
          c7       0.95      0.99      0.97        70
          c8       0.99      0.85      0.91        84
          c9       0.97      0.95      0.96        79

    accuracy                           0.95       800
   macro avg       0.95      0.95      0.95       800
weighted avg       0.95      0.95      0.95       800



Both models are created now. Let's evaluate on the small labeled test set.

In [55]:
os.getcwd()

'C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\raw'

In [56]:
test_labeled_df = pd.read_csv('../processed/labeled_test_df.csv')

In [80]:
test_labeled_df

,filename,classname
0,img_101570.jpg,c0
1,img_102035.jpg,c0
2,img_22.jpg,c0
3,img_23.jpg,c0
4,img_24.jpg,c0
...,...,...
195,img_54733.jpg,c9
196,img_54928.jpg,c9
197,img_54990.jpg,c9
198,img_55171.jpg,c9


In [81]:
os.getcwd()

'C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\raw\\imgs\\testlabeled\\unsorted'

In [82]:
os.chdir('imgs/testlabeled/unsorted')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'imgs/testlabeled/unsorted'

In [83]:
test = test_dgen.flow_from_dataframe(test_labeled_df,
                                     x_col='filename',
                                     y_col='classname',
                                     target_size=(227,227),
                                     batch_size=16,
                                     shuffle=False)

Found 200 validated image filenames belonging to 10 classes.


In [84]:
test_nn_output = model3.predict(test)

In [85]:
model3.evaluate(test)

  ...
    to  
  ['...']
13/13 [==============================] - 2s 125ms/step - loss: 0.8906 - acc: 0.7300


[0.8905925131761111, 0.73]

In [86]:
y_labeled_test=test_labeled_df[['classname']]

In [87]:
X_labeled_test = pd.DataFrame(test_nn_output, 
                              columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [88]:
X_labeled_test

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,0.047275,0.005121,0.027903,0.069816,0.096730,0.443145,0.157516,0.006203,0.106239,0.040052
1,0.757133,0.000159,0.000689,0.019632,0.008444,0.082580,0.000070,0.000231,0.001939,0.129122
2,0.393718,0.007405,0.000747,0.249553,0.039259,0.031297,0.001105,0.001983,0.012218,0.262715
3,0.469957,0.009404,0.002149,0.127384,0.066894,0.024054,0.001107,0.005694,0.015252,0.278104
4,0.102190,0.102216,0.010231,0.464914,0.107785,0.008216,0.053829,0.014488,0.033943,0.102189
...,...,...,...,...,...,...,...,...,...,...
195,0.004901,0.107628,0.026312,0.000329,0.000066,0.002584,0.002301,0.498552,0.013659,0.343667
196,0.001136,0.006047,0.019866,0.002046,0.004147,0.001085,0.005265,0.003656,0.944786,0.011967
197,0.486174,0.046821,0.000994,0.012781,0.000907,0.115279,0.001817,0.001643,0.005545,0.328040
198,0.106881,0.007360,0.058115,0.084603,0.080829,0.007823,0.006192,0.143699,0.163061,0.341437


In [89]:
y_labeled_test

,classname
0,c0
1,c0
2,c0
3,c0
4,c0
...,...
195,c9
196,c9
197,c9
198,c9


In [94]:
labeled_predictions = RandForest.predict(X_labeled_test)

In [95]:
labeled_predictions

array(['c6', 'c0', 'c0', 'c0', 'c3', 'c0', 'c0', 'c0', 'c0', 'c0', 'c5',
       'c0', 'c0', 'c0', 'c0', 'c4', 'c0', 'c0', 'c0', 'c0', 'c1', 'c1',
       'c1', 'c1', 'c1', 'c8', 'c2', 'c1', 'c1', 'c6', 'c1', 'c8', 'c1',
       'c1', 'c6', 'c1', 'c1', 'c2', 'c1', 'c1', 'c6', 'c6', 'c8', 'c2',
       'c1', 'c7', 'c1', 'c1', 'c2', 'c6', 'c7', 'c2', 'c2', 'c8', 'c1',
       'c7', 'c8', 'c2', 'c2', 'c2', 'c3', 'c5', 'c3', 'c4', 'c3', 'c3',
       'c3', 'c3', 'c4', 'c4', 'c3', 'c3', 'c3', 'c3', 'c3', 'c3', 'c3',
       'c3', 'c0', 'c3', 'c8', 'c3', 'c4', 'c4', 'c8', 'c4', 'c4', 'c4',
       'c4', 'c4', 'c3', 'c3', 'c4', 'c4', 'c4', 'c4', 'c4', 'c3', 'c4',
       'c4', 'c5', 'c5', 'c5', 'c5', 'c5', 'c5', 'c5', 'c5', 'c5', 'c5',
       'c5', 'c5', 'c5', 'c5', 'c8', 'c5', 'c5', 'c5', 'c5', 'c5', 'c6',
       'c6', 'c6', 'c6', 'c6', 'c1', 'c6', 'c6', 'c6', 'c6', 'c6', 'c1',
       'c8', 'c8', 'c6', 'c8', 'c6', 'c6', 'c6', 'c6', 'c7', 'c7', 'c7',
       'c8', 'c8', 'c8', 'c7', 'c8', 'c1', 'c7', 'c

In [96]:
print(classification_report(y_labeled_test, labeled_predictions))

              precision    recall  f1-score   support

          c0       0.70      0.80      0.74        20
          c1       0.61      0.70      0.65        20
          c2       0.78      0.35      0.48        20
          c3       0.71      0.75      0.73        20
          c4       0.78      0.70      0.74        20
          c5       0.86      0.95      0.90        20
          c6       0.65      0.75      0.70        20
          c7       0.73      0.55      0.63        20
          c8       0.42      0.80      0.55        20
          c9       1.00      0.40      0.57        20

    accuracy                           0.68       200
   macro avg       0.72      0.68      0.67       200
weighted avg       0.72      0.68      0.67       200



In [97]:
print(RandForest.predict_proba(X_labeled_test))

[[0.01       0.006      0.04404762 ... 0.002      0.1195     0.01      ]
 [0.90940476 0.         0.         ... 0.01       0.04083333 0.0225    ]
 [0.94166667 0.         0.         ... 0.         0.         0.05833333]
 ...
 [0.92666667 0.         0.         ... 0.         0.         0.06      ]
 [0.2202619  0.00666667 0.0205     ... 0.015      0.17980952 0.36592857]
 [0.11433333 0.00916667 0.0165     ... 0.         0.022      0.6155    ]]


Load datagen for actual test data

In [49]:
test_filenames = os.listdir('data/raw/imgs/test/test')

In [50]:
test_filenames = pd.DataFrame({'filename':test_filenames})

In [51]:
test_filenames

,filename
0,img_1.jpg
1,img_10.jpg
2,img_100.jpg
3,img_1000.jpg
4,img_100000.jpg
...,...
79721,img_99994.jpg
79722,img_99995.jpg
79723,img_99996.jpg
79724,img_99998.jpg


In [52]:
test_filenames.to_csv('data/raw/test_filenames.csv', index=False)

In [59]:
df_test

,filename
0,img_1.jpg
1,img_10.jpg
2,img_100.jpg
3,img_1000.jpg
4,img_100000.jpg
...,...
79721,img_99994.jpg
79722,img_99995.jpg
79723,img_99996.jpg
79724,img_99998.jpg


In [60]:
df_test['filename'] = df_test[['filename']].apply(lambda x: 'C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/raw/imgs/test/test/' + x, axis=1)

In [61]:
df_test

,filename
0,C:/Users/Dylan/Desktop/Data Science/Projects/D...
1,C:/Users/Dylan/Desktop/Data Science/Projects/D...
2,C:/Users/Dylan/Desktop/Data Science/Projects/D...
3,C:/Users/Dylan/Desktop/Data Science/Projects/D...
4,C:/Users/Dylan/Desktop/Data Science/Projects/D...
...,...
79721,C:/Users/Dylan/Desktop/Data Science/Projects/D...
79722,C:/Users/Dylan/Desktop/Data Science/Projects/D...
79723,C:/Users/Dylan/Desktop/Data Science/Projects/D...
79724,C:/Users/Dylan/Desktop/Data Science/Projects/D...


In [105]:
os.getcwd()

'C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\raw\\imgs\\test\\test'

In [62]:
df_test['class'] = 'unknown'

In [64]:
df_test.to_csv('C:/Users/Dylan/Desktop/Data Science/Projects/DistractedDrivers/data/raw/test_filenames.csv', index=False)

In [107]:
test = test_dgen.flow_from_dataframe(test_filenames,
                                     target_size=(227,227),
                                     batch_size=16,
                                     shuffle=False)

Found 79726 validated image filenames belonging to 1 classes.


In [108]:
test_predictions = model3.predict(test)

In [109]:
test_predictions = pd.DataFrame(test_predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [110]:
test_predictions

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,0.008604,0.085207,0.018095,0.113947,0.040185,0.600935,0.077657,0.006483,0.026404,0.022482
1,0.019094,0.000826,0.001663,0.006799,0.003642,0.941262,0.001846,0.000832,0.017235,0.006800
2,0.200818,0.249065,0.058669,0.010818,0.004579,0.048115,0.014242,0.062749,0.039698,0.311247
3,0.004387,0.002688,0.024336,0.000805,0.002280,0.009872,0.004561,0.000767,0.943617,0.006687
4,0.045187,0.001458,0.000840,0.701084,0.199329,0.022523,0.005878,0.001054,0.016436,0.006211
...,...,...,...,...,...,...,...,...,...,...
79721,0.004477,0.186477,0.133851,0.001749,0.000536,0.011587,0.029475,0.449997,0.096771,0.085079
79722,0.013295,0.035097,0.000265,0.931134,0.004813,0.001793,0.002391,0.000746,0.002524,0.007941
79723,0.000791,0.041994,0.000147,0.846906,0.099453,0.000325,0.007390,0.000138,0.002115,0.000740
79724,0.000449,0.085182,0.286448,0.000522,0.000433,0.000592,0.328923,0.205457,0.084118,0.007875


In [121]:
test_predictions_df = test_filenames.join(test_predictions)
test_predictions_df.drop(columns=['class'], inplace=True)
test_predictions_df.rename(columns={'filename':'img'}, inplace=True)
test_predictions_df

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.008604,0.085207,0.018095,0.113947,0.040185,0.600935,0.077657,0.006483,0.026404,0.022482
1,img_10.jpg,0.019094,0.000826,0.001663,0.006799,0.003642,0.941262,0.001846,0.000832,0.017235,0.006800
2,img_100.jpg,0.200818,0.249065,0.058669,0.010818,0.004579,0.048115,0.014242,0.062749,0.039698,0.311247
3,img_1000.jpg,0.004387,0.002688,0.024336,0.000805,0.002280,0.009872,0.004561,0.000767,0.943617,0.006687
4,img_100000.jpg,0.045187,0.001458,0.000840,0.701084,0.199329,0.022523,0.005878,0.001054,0.016436,0.006211
...,...,...,...,...,...,...,...,...,...,...,...
79721,img_99994.jpg,0.004477,0.186477,0.133851,0.001749,0.000536,0.011587,0.029475,0.449997,0.096771,0.085079
79722,img_99995.jpg,0.013295,0.035097,0.000265,0.931134,0.004813,0.001793,0.002391,0.000746,0.002524,0.007941
79723,img_99996.jpg,0.000791,0.041994,0.000147,0.846906,0.099453,0.000325,0.007390,0.000138,0.002115,0.000740
79724,img_99998.jpg,0.000449,0.085182,0.286448,0.000522,0.000433,0.000592,0.328923,0.205457,0.084118,0.007875


In [126]:
test_predictions_df.to_csv('C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\test_predictions(VGG16_Transfer_Learning).csv', index=False)

In [111]:
rf_predictions = model.predict_proba(test_predictions)

In [112]:
rf_predictions

array([[0.01277273, 0.05188699, 0.06216909, ..., 0.0022028 , 0.07507054,
        0.02621848],
       [0.        , 0.        , 0.        , ..., 0.        , 0.0188447 ,
        0.00457516],
       [0.50327548, 0.19732263, 0.00961905, ..., 0.        , 0.02691071,
        0.22182527],
       ...,
       [0.00173611, 0.        , 0.        , ..., 0.        , 0.00291667,
        0.        ],
       [0.        , 0.        , 0.19470271, ..., 0.38539343, 0.19947546,
        0.03235011],
       [0.        , 0.        , 0.00482759, ..., 0.00206897, 0.0069697 ,
        0.01766667]])

In [113]:
rf_test_predictions = pd.DataFrame(rf_predictions, columns=['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])

In [114]:
predictions = test_filenames.join(rf_test_predictions)

In [115]:
predictions.drop(columns=['class'], inplace=True)

In [116]:
predictions.rename(columns={'filename': 'img'}, inplace=True)

In [117]:
predictions

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.012773,0.051887,0.062169,0.045604,0.075507,0.612750,0.035819,0.002203,0.075071,0.026218
1,img_10.jpg,0.000000,0.000000,0.000000,0.000000,0.011932,0.964648,0.000000,0.000000,0.018845,0.004575
2,img_100.jpg,0.503275,0.197323,0.009619,0.003040,0.000000,0.024722,0.013285,0.000000,0.026911,0.221825
3,img_1000.jpg,0.000526,0.001600,0.003887,0.003750,0.005671,0.037143,0.023430,0.010000,0.770966,0.143027
4,img_100000.jpg,0.006813,0.000000,0.000000,0.957205,0.029316,0.001250,0.000000,0.000000,0.005417,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
79721,img_99994.jpg,0.028851,0.258388,0.141071,0.000000,0.000000,0.009987,0.022551,0.240912,0.170321,0.127919
79722,img_99995.jpg,0.012290,0.010714,0.021952,0.854877,0.000000,0.039625,0.010625,0.022601,0.012248,0.015068
79723,img_99996.jpg,0.001736,0.000000,0.000000,0.991806,0.001667,0.001250,0.000625,0.000000,0.002917,0.000000
79724,img_99998.jpg,0.000000,0.000000,0.194703,0.000000,0.000000,0.001304,0.186774,0.385393,0.199475,0.032350


In [118]:
os.getcwd()

'C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\raw\\imgs\\test\\test'

In [127]:
predictions.to_csv('C:\\Users\\Dylan\\Desktop\\Data Science\\Projects\\DistractedDrivers\\data\\test_predictions(VGG16_Transfer_Learning_with_RandFor).csv', index=False)

In [124]:
test_predictions_df

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.008604,0.085207,0.018095,0.113947,0.040185,0.600935,0.077657,0.006483,0.026404,0.022482
1,img_10.jpg,0.019094,0.000826,0.001663,0.006799,0.003642,0.941262,0.001846,0.000832,0.017235,0.006800
2,img_100.jpg,0.200818,0.249065,0.058669,0.010818,0.004579,0.048115,0.014242,0.062749,0.039698,0.311247
3,img_1000.jpg,0.004387,0.002688,0.024336,0.000805,0.002280,0.009872,0.004561,0.000767,0.943617,0.006687
4,img_100000.jpg,0.045187,0.001458,0.000840,0.701084,0.199329,0.022523,0.005878,0.001054,0.016436,0.006211
...,...,...,...,...,...,...,...,...,...,...,...
79721,img_99994.jpg,0.004477,0.186477,0.133851,0.001749,0.000536,0.011587,0.029475,0.449997,0.096771,0.085079
79722,img_99995.jpg,0.013295,0.035097,0.000265,0.931134,0.004813,0.001793,0.002391,0.000746,0.002524,0.007941
79723,img_99996.jpg,0.000791,0.041994,0.000147,0.846906,0.099453,0.000325,0.007390,0.000138,0.002115,0.000740
79724,img_99998.jpg,0.000449,0.085182,0.286448,0.000522,0.000433,0.000592,0.328923,0.205457,0.084118,0.007875


In [125]:
predictions

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.012773,0.051887,0.062169,0.045604,0.075507,0.612750,0.035819,0.002203,0.075071,0.026218
1,img_10.jpg,0.000000,0.000000,0.000000,0.000000,0.011932,0.964648,0.000000,0.000000,0.018845,0.004575
2,img_100.jpg,0.503275,0.197323,0.009619,0.003040,0.000000,0.024722,0.013285,0.000000,0.026911,0.221825
3,img_1000.jpg,0.000526,0.001600,0.003887,0.003750,0.005671,0.037143,0.023430,0.010000,0.770966,0.143027
4,img_100000.jpg,0.006813,0.000000,0.000000,0.957205,0.029316,0.001250,0.000000,0.000000,0.005417,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
79721,img_99994.jpg,0.028851,0.258388,0.141071,0.000000,0.000000,0.009987,0.022551,0.240912,0.170321,0.127919
79722,img_99995.jpg,0.012290,0.010714,0.021952,0.854877,0.000000,0.039625,0.010625,0.022601,0.012248,0.015068
79723,img_99996.jpg,0.001736,0.000000,0.000000,0.991806,0.001667,0.001250,0.000625,0.000000,0.002917,0.000000
79724,img_99998.jpg,0.000000,0.000000,0.194703,0.000000,0.000000,0.001304,0.186774,0.385393,0.199475,0.032350
